In [1]:
from requests_html import HTMLSession, HTML
import requests
import json
import time
import pandas as pd
import datetime as dt
import re
from selenium import webdriver

In [2]:
#Hàm crawler dữ liệu từ Phong Vũ
#Input: -url của hãng laptop ta muốn crawl
#       -selenium webdriver của trình duyệt ta sử dụng
#Output: Dataframe ta crawl được từ Phong Vũ
def CrawlPhongVu(url, browser):  
    df = pd.DataFrame([], columns = ['Name', 'Price'])  
    count = 1  
    while 1:
        u = url+'&p='+str(count)
        print(u)
        browser.get(u)
        time.sleep(1)
        html_text = HTML(html=browser.page_source)
        post = html_text.find('.product-item')
        if len(post) == 0:
            break
        for field in post:
            name = field.find('.product-name')[0].text
            try:
                price = field.find('.product-price')[0].text
                if '\n' in price:
                    price = price.split('\n')[1]
            except IndexError:
                price = 0
            df2 = pd.DataFrame([[name, price]], columns = ['Name', 'Price']) 
            df = df.append(df2, ignore_index=True)
        time.sleep(1)
        count += 1
    return df

In [13]:
#Danh sách url của các hãng ta sẽ crawl
urllist = [
    'https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer',
    'https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus',
    'https://phongvu.vn/laptop-Dell-1056.cat?pv_medium=m-laptop-Dell',
    'https://phongvu.vn/laptop-hp-1057.cat?pv_medium=m-laptop-hp',
    'https://phongvu.vn/laptop-lenovo-1060.cat?pv_medium=m-laptop-lenovo',
    'https://phongvu.vn/laptop-lg-1268.cat?pv_medium=m-laptop-lg',
    'https://phongvu.vn/macbook-1061.cat?pv_medium=m-macbook',
    'https://phongvu.vn/laptop-msi-1081.cat?pv_medium=m-laptop-msi',
]

In [4]:
#Bắt đầu quá trình crawl
df = pd.DataFrame([], columns = ['Name', 'Price'])
browser = webdriver.Chrome(executable_path='chromedriver.exe')
for url in urllist:
    df2 = CrawlPhongVu(url, browser)
    df = df.append(df2, ignore_index=True)
df.head()

https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=1
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=2
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=3
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=4
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=5
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=6
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=7
https://phongvu.vn/laptop-acer-1059.cat?pv_medium=m-laptop-acer&p=8
https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus&p=1
https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus&p=2
https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus&p=3
https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus&p=4
https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus&p=5
https://phongvu.vn/laptop-asus-1058.cat?pv_medium=m-laptop-asus&p=6
https://phongvu.vn/laptop-asus-1058.cat?pv_mediu

,Name,Price
0,Laptop Acer Aspire 5 A514-52-33AB (NX.HMHSV.00...,11.990.000đ
1,Laptop Acer Aspire 3 A315-54K-36QU (NX.HEESV.0...,9.990.000đ
2,Laptop Acer Swift 3 SF315-52-52Z7 (NX.GZBSV.00...,15.999.000đ
3,Laptop Acer Swift 3 SF315-52-38YQ (NX.GZBSV.00...,13.499.000đ
4,Laptop Acer Swift 3 SF314-56-38UE (NX.H4CSV.00...,14.499.000đ


In [5]:
#Số laptop crawl được từ Phong Vũ
len(df)

726

In [6]:
#Loại bỏ các sản phẩm có cột Price trống do sản phẩm không còn bán
for i in range(len(df)):
    if df['Price'][i] == 0:
        df = df.drop(i)
len(df)

665

Dữ liệu crawl về sẽ có 2 cột là Name và Price

Name: Chứa tên của laptop và thông tin cấu hình của máy với format <br/>
(Kích thước màn hình" Loại màn hình/Thông tin CPU/Ram/Thông tin ổ chứa/GPU/Hệ điều hành/Cân nặng)

Price: Chứa chuỗi thể hiện giá tiền của laptop với format XXX.XXX.XXXđ

In [9]:
#Ví dụ cột Name
df['Name'][586]

'Laptop Lenovo ThinkBook 13s-IWL-20R900DHVN (13" FHD/i5-8265U/8GB/256GB SSD/UHD 620/Win10/1.3 kg)'

In [10]:
#df = df.dropna()
#Ta reset index của DataFrame do ta vừa bỏ những dòng không có giá tiền ở trên
df = df.reset_index(drop=True)

In [11]:
#Bỏ những dòng có Name không tuân thủ luật format bên trên
for i in range(len(df)):
    s = df['Name'][i]
    n = len(s.split('(')[-1].strip(')').split('/'))
    if n != 7:
        df = df.drop(i)    

In [12]:
#Ta reset index của DataFrame do ta vừa bỏ những dòng không tuân thủ luật
df = df.reset_index(drop=True)
len(df)

535

In [14]:
#Lưu lại dữ liệu
df.to_csv(path_or_buf='data_laptop.csv', index=False, sep='\t')

In [2]:
#Đọc lại dữ liệu
df = pd.read_csv("data_laptop.csv",sep='\t')
#df = df.drop(['level_0', 'index'], axis=1)
df.head()

,Name,Price
0,Laptop Acer Aspire 5 A514-52-33AB (NX.HMHSV.00...,11.990.000đ
1,Laptop Acer Aspire 3 A315-54K-36QU (NX.HEESV.0...,9.990.000đ
2,Laptop Acer Swift 3 SF315-52-52Z7 (NX.GZBSV.00...,15.999.000đ
3,Laptop Acer Swift 3 SF315-52-38YQ (NX.GZBSV.00...,13.499.000đ
4,Laptop Acer Swift 3 SF314-56-38UE (NX.H4CSV.00...,14.499.000đ


In [3]:
#Hàm thêm field cho Dataframe
def AddField(df):
    b = [-1]*len(df)
    #Nhãn hiệu
    df['Brand'] = b
    #Kích thước màn hình
    df['Screen Size'] = b
    #Loại màn hình
    df['Screen Type'] = b
    #Hãng CPU
    df['CPU Brand'] = b
    #Loại CPU vd:i3, i5,..
    df['CPU Type'] = b
    #Thế hệ CPU
    df['CPU Gen'] = b
    #Ram
    df['Ram'] = b
    #Dung lượng
    df['Capacity'] = b
    #Loại ổ lưu trữ
    df['Storage Type'] = b
    #Card đồ họa
    df['GPU'] = b
    #Hệ điều hành
    df['OS'] = b
    #Cân nặng
    df['Weight'] = [-1.0]*len(df)
    return df

In [4]:
df = AddField(df)

In [5]:
#Trích xuất thông tin từ cột Name
for i in range(len(df)):
    s = df['Name'][i]
    a = s.split()
    df['Brand'][i] = a[1].lower()
    res = [i.start() for i in re.finditer('\(', s)] 
    df['Name'][i] = s[s.index(a[2]):res[-1]-1]

    buffer = s.split('(')[-1].strip(')').lower().split('/')
    df['Screen Size'][i] = float(buffer[0].split('"')[0].replace('\'', ''))
    if df['Brand'][i] == 'apple':
        df['Screen Type'][i] = 'qhd'
    else:
        try:
            df['Screen Type'][i] = buffer[0].split('"')[1].replace(' ', '')
        except:
            df['Screen Type'][i] = 'hd'
    if buffer[1][0] == 'r':
        df['CPU Brand'][i] = 'amd'
        arr = re.findall(r'[0-9]+', buffer[1])
        df['CPU Type'][i] = int(arr[0])
        df['CPU Gen'][i] = int(arr[1][0])
    elif buffer[1][0] == '2':
        df['CPU Brand'][i] = 'amd'
        df['CPU Type'][i] = buffer[1][1]
        df['CPU Gen'][i] = 2
    else:
        df['CPU Brand'][i] = 'intel'
        arr = re.findall(r'[0-9]+', buffer[1])
        if df['Brand'][i] == 'apple':
            df['CPU Type'][i] = 5
            df['CPU Gen'][i] = 8
        elif len(arr) == 1:
            df['CPU Type'][i] = 1
            df['CPU Gen'][i] = int(arr[0][0])
        else:
            df['CPU Type'][i] = int(arr[0])
            df['CPU Gen'][i] = int(arr[1][0])
    df['Ram'][i] = int(buffer[2][0])
    arr = re.findall(r'[0-9]+', buffer[3])
    if len(arr[0]) == 1:
        df['Capacity'][i] = int(arr[0])*1000
    else:
        df['Capacity'][i] = int(arr[0])
    try:
        df['Storage Type'][i] = buffer[3].split()[1]
    except:
        if df['Capacity'][i] >= 1000:           
            df['Storage Type'][i] = 'hdd'
        else:
            df['Storage Type'][i] = 'ssd'
    df['GPU'][i] = buffer[4]
    df['OS'][i] = buffer[5]
    df['Weight'][i] = float(buffer[6].strip('kg').strip(' '))
    print(i)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.

0
1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
3
4
5
6
7
8


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

212
213
214


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534


In [6]:
df.head(10)

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
0,Aspire 5 A514-52-33AB (NX.HMHSV.001),11.990.000đ,acer,14,fhd,intel,3,1,4,256,ssd,uhd 620,win10,1.6
1,Aspire 3 A315-54K-36QU (NX.HEESV.007),9.990.000đ,acer,15,fhd,intel,3,7,4,256,ssd,hd 620,win10,1.7
2,Swift 3 SF315-52-52Z7 (NX.GZBSV.004),15.999.000đ,acer,15,fhd,intel,5,8,4,1000,hdd,uhd 620,win10,1.6
3,Swift 3 SF315-52-38YQ (NX.GZBSV.003),13.499.000đ,acer,15,fhd,intel,3,8,4,1000,hdd,uhd 620,win10,1.6
4,Swift 3 SF314-56-38UE (NX.H4CSV.005),14.499.000đ,acer,14,fhd,intel,3,8,4,256,ssd,uhd 620,win10,1.5
5,Nitro 5 AN515-54-59SF (NH.Q5ASV.013),22.490.000đ,acer,15,fhd,intel,5,9,8,512,ssd,gtx 1050,win10,2.3
6,Aspire 5 A515-54-54EU (NX.HN3SV.002),15.990.000đ,acer,15,fhd,intel,5,1,8,512,ssd,uhd 620,win10,1.7
7,Aspire 5 A515-54G-51J3 (NX.HN5SV.003),17.990.000đ,acer,15,fhd,intel,5,1,8,1000,ssd,uhd 620,win10,1.7
8,Nitro 5 AN515-54-51X1 (NH.Q5ASV.011),21.990.000đ,acer,15,fhd,intel,5,9,8,256,ssd,gtx 1050,win10,2.3
9,Nitro 5 AN515-43-R84R (NH.Q5XSV.001),18.999.000đ,acer,15,fhd,amd,5,3,8,256,ssd,radeon rx 560x,win10,2.3


In [7]:
#Lưu lại dữ liệu vừa xử lý
df.to_csv(path_or_buf='data_laptop.csv', index=False, sep='\t')

In [40]:
#Gọi lại dữ liệu
df = pd.read_csv("data_laptop.csv",sep='\t')
#df = df.drop(['index'], axis = 1)
df.head(10)

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
0,Aspire 5 A514-52-33AB (NX.HMHSV.001),11.990.000đ,acer,14,fhd,intel,3,1,4,256,ssd,uhd 620,win10,1.6
1,Aspire 3 A315-54K-36QU (NX.HEESV.007),9.990.000đ,acer,15,fhd,intel,3,7,4,256,ssd,hd 620,win10,1.7
2,Swift 3 SF315-52-52Z7 (NX.GZBSV.004),15.999.000đ,acer,15,fhd,intel,5,8,4,1000,hdd,uhd 620,win10,1.6
3,Swift 3 SF315-52-38YQ (NX.GZBSV.003),13.499.000đ,acer,15,fhd,intel,3,8,4,1000,hdd,uhd 620,win10,1.6
4,Swift 3 SF314-56-38UE (NX.H4CSV.005),14.499.000đ,acer,14,fhd,intel,3,8,4,256,ssd,uhd 620,win10,1.5
5,Nitro 5 AN515-54-59SF (NH.Q5ASV.013),22.490.000đ,acer,15,fhd,intel,5,9,8,512,ssd,gtx 1050,win10,2.3
6,Aspire 5 A515-54-54EU (NX.HN3SV.002),15.990.000đ,acer,15,fhd,intel,5,1,8,512,ssd,uhd 620,win10,1.7
7,Aspire 5 A515-54G-51J3 (NX.HN5SV.003),17.990.000đ,acer,15,fhd,intel,5,1,8,1000,ssd,uhd 620,win10,1.7
8,Nitro 5 AN515-54-51X1 (NH.Q5ASV.011),21.990.000đ,acer,15,fhd,intel,5,9,8,256,ssd,gtx 1050,win10,2.3
9,Nitro 5 AN515-43-R84R (NH.Q5XSV.001),18.999.000đ,acer,15,fhd,amd,5,3,8,256,ssd,radeon rx 560x,win10,2.3


Sau đó ta bắt đầu kiểm tra từng field xem trong quá trình trích xuất thông tin có gặp vấn đề gì không

In [41]:
#Kiểm tra Field Brand
df.groupby('Brand').count()

,Name,Price,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
Brand,,,,,,,,,,,,,
acer,89,89,89,89,89,89,89,89,89,89,89,89,89
apple,18,18,18,18,18,18,18,18,18,18,18,18,18
asus,163,163,163,162,163,163,163,163,163,163,163,163,163
dell,69,69,69,69,69,69,69,69,69,69,69,69,69
hp,112,112,112,112,112,112,112,112,112,112,112,112,112
lenovo,55,55,55,54,55,55,55,55,55,55,55,55,55
lg,1,1,1,1,1,1,1,1,1,1,1,1,1
msi,27,27,27,27,27,27,27,27,27,27,27,27,27
swift,1,1,1,1,1,1,1,1,1,1,1,1,1


In [42]:
#Quan sát dữ liệu trên ta thấy Swift không phải 1 hãng máy tính nhưng lại nằm trong Brand
display(df.loc[df['Brand'] == 'swift'])
#Vào trang chủ tìm kiếm thì rõ được vấn đề là người nhập liệu nhập thiếu hãng Acer trong tên
df.loc[df['Brand'] == 'swift', 'Brand'] = 'acer'

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
61,3 SF315-52G-87N4 (NX.GZCSV.005),23.590.000đ,swift,15,fhd,intel,7,8,8,128,ssd,mx150,win10,1.7


In [43]:
#Ta thấy hãng LG chỉ có duy nhất 1 máy, quá ít để học nên ta sẽ xóa luôn hãng này
df = df.loc[df['Brand'] != 'lg']
df = df.reset_index(drop=True)
len(df)

534

In [44]:
#Quan sát bảng trên ta thấy được Sreen Type của 1 máy lenovo và 1 máy asus bị thiếu (do người nhập liệu nhập thiếu)
df[df.isna().any(axis=1)]

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
221,VivoBook S S410UN-EB022T,19.490.000đ,asus,14,NaN,intel,5,8,4,128,ssd,nvidia geforce mx150 2gb,windows 10 home sl 64-bit,1.4
488,Ideapad 530s-14IKB 81EU007QVN,16.590.000đ,lenovo,14,NaN,intel,5,8,4,256,ssd,intel uhd graphics 620,windows 10 home sl 64-bit,1.4


In [45]:
#Sau tìm hiểu thì cả 2 máy đều có màn hình hd
df.loc[df.isna().any(axis=1), 'Screen Type'] = 'hd'

In [46]:
#Kiểm tra Field Screen Size
df.groupby('Screen Size').count()

,Name,Price,Brand,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
Screen Size,,,,,,,,,,,,,
11,1,1,1,1,1,1,1,1,1,1,1,1,1
13,65,65,65,65,65,65,65,65,65,65,65,65,65
14,176,176,176,176,176,176,176,176,176,176,176,176,176
15,283,283,283,283,283,283,283,283,283,283,283,283,283
17,9,9,9,9,9,9,9,9,9,9,9,9,9


In [47]:
#Kiểm tra Field Screen Type
df.groupby('Screen Type').count()

,Name,Price,Brand,Screen Size,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
Screen Type,,,,,,,,,,,,,
fhd,409,409,409,409,409,409,409,409,409,409,409,409,409
hd,107,107,107,107,107,107,107,107,107,107,107,107,107
qhd,18,18,18,18,18,18,18,18,18,18,18,18,18


In [48]:
#Kiểm tra Field CPU Brand
df.groupby('CPU Brand').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
CPU Brand,,,,,,,,,,,,,
amd,28,28,28,28,28,28,28,28,28,28,28,28,28
intel,506,506,506,506,506,506,506,506,506,506,506,506,506


In [49]:
#Kiểm tra Field CPU Type
df.groupby('CPU Type').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
CPU Type,,,,,,,,,,,,,
1,14,14,14,14,14,14,14,14,14,14,14,14,14
2,1,1,1,1,1,1,1,1,1,1,1,1,1
3,118,118,118,118,118,118,118,118,118,118,118,118,118
4,2,2,2,2,2,2,2,2,2,2,2,2,2
5,271,271,271,271,271,271,271,271,271,271,271,271,271
7,126,126,126,126,126,126,126,126,126,126,126,126,126
9,2,2,2,2,2,2,2,2,2,2,2,2,2


In [50]:
#Có những điểm nghi ngờ vì thường CPU Type sẽ có số lẻ
display(df.loc[df['CPU Type'] % 2 == 0])

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
238,VivoBook X505ZA-EJ493T,9.990.000đ,asus,15,fhd,amd,2,2,4,1000,hdd,radeon vega 3,win10,1.7
477,Ideapad 330-14AST (81D5002CVN),7.290.000đ,lenovo,14,hd,intel,4,9,4,500,hdd,radeon r3,win10,2.1
485,Ideapad 130-15AST (81H50020VN),6.790.000đ,lenovo,15,hd,intel,4,9,4,500,hdd,radeon r3,win10,2.1


In [51]:
#Sau khi tìm hiểu thì CPU Type của AMD sẽ có thể có số chẵn 
df.loc[df['CPU Type'] % 2 == 0, 'CPU Brand'] = 'amd'

In [52]:
#Kiểm tra Field CPU Gen
df.groupby('CPU Gen').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,Ram,Capacity,Storage Type,GPU,OS,Weight
CPU Gen,,,,,,,,,,,,,
1,24,24,24,24,24,24,24,24,24,24,24,24,24
2,11,11,11,11,11,11,11,11,11,11,11,11,11
3,20,20,20,20,20,20,20,20,20,20,20,20,20
4,6,6,6,6,6,6,6,6,6,6,6,6,6
5,6,6,6,6,6,6,6,6,6,6,6,6,6
6,2,2,2,2,2,2,2,2,2,2,2,2,2
7,39,39,39,39,39,39,39,39,39,39,39,39,39
8,373,373,373,373,373,373,373,373,373,373,373,373,373
9,53,53,53,53,53,53,53,53,53,53,53,53,53


In [53]:
#Có những điểm nghi ngờ vì cả AMD và Intel đều không có chip thế hệ thứ 1
display(df.loc[df['CPU Gen'] == 1])
#Hóa ra đó là chip thế hệ thứ 10
df.loc[df['CPU Gen'] == 1, 'CPU Gen'] = 10

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
0,Aspire 5 A514-52-33AB (NX.HMHSV.001),11.990.000đ,acer,14,fhd,intel,3,1,4,256,ssd,uhd 620,win10,1.6
6,Aspire 5 A515-54-54EU (NX.HN3SV.002),15.990.000đ,acer,15,fhd,intel,5,1,8,512,ssd,uhd 620,win10,1.7
7,Aspire 5 A515-54G-51J3 (NX.HN5SV.003),17.990.000đ,acer,15,fhd,intel,5,1,8,1000,ssd,uhd 620,win10,1.7
30,Aspire 3 A315-54-368N (NX.HM2SV.004),11.990.000đ,acer,15,fhd,intel,3,1,8,512,ssd,intel uhd,win10,1.7
36,Aspire 3 A315-54-59ZJ (NX.HM2SV.005),14.490.000đ,acer,15,fhd,intel,5,1,8,512,ssd,intel uhd,win10,1.7
38,Aspire 3 A315-54-36QY (NX.HM2SV.001),10.990.000đ,acer,15,fhd,intel,3,1,4,256,ssd,intel uhd,win10,1.7
43,Aspire 3 A315-54-52HT (NX.HM2SV.002),12.990.000đ,acer,15,fhd,intel,5,1,4,256,ssd,intel uhd,win10,1.7
50,Swift 3 SF314-57G-53T1 (NX.HJESV.001),19.990.000đ,acer,14,fhd,intel,5,1,8,512,ssd,mx250,win10,1.2
53,Swift 3 SF314-57-52GB (NX.HJFSV.001),18.990.000đ,acer,14,fhd,intel,5,1,8,512,ssd,intel uhd,win10,1.2
56,Swift 3 SF314-57-54B2 (NX.HJKSV.001),18.990.000đ,acer,14,fhd,intel,5,1,8,512,ssd,intel uhd,win10,1.2


In [54]:
#Kiểm tra Field Ram
df.groupby('Ram').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Capacity,Storage Type,GPU,OS,Weight
Ram,,,,,,,,,,,,,
1,28,28,28,28,28,28,28,28,28,28,28,28,28
4,287,287,287,287,287,287,287,287,287,287,287,287,287
8,219,219,219,219,219,219,219,219,219,219,219,219,219


In [55]:
#Rõ ràng ram 16gb đã bị nhầm thành ram 1gb
df.loc[df['Ram'] == 1, 'Ram'] = 16

In [56]:
#Kiểm tra Field Capacity
df.groupby('Capacity').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Storage Type,GPU,OS,Weight
Capacity,,,,,,,,,,,,,
16,3,3,3,3,3,3,3,3,3,3,3,3,3
64,2,2,2,2,2,2,2,2,2,2,2,2,2
128,13,13,13,13,13,13,13,13,13,13,13,13,13
256,117,117,117,117,117,117,117,117,117,117,117,117,117
500,31,31,31,31,31,31,31,31,31,31,31,31,31
512,105,105,105,105,105,105,105,105,105,105,105,105,105
1000,259,259,259,259,259,259,259,259,259,259,259,259,259
2000,4,4,4,4,4,4,4,4,4,4,4,4,4


In [57]:
#Capacity bằng 16 thật ra chỉ là dung lượng ổ Optane
df.loc[df['Capacity'] == 16]

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
13,Aspire 5 A515-53-50ZD (NX.H6DSV.001),14.990.000đ,acer,15,hd,intel,5,8,4,16,intel,1tb hdd,win10,2.0
15,Aspire 5 A514-51-58ZJ (NX.H6XSV.001),14.990.000đ,acer,14,hd,intel,5,8,4,16,intel,1tb hdd,win10,1.9
16,Aspire 5 A515-53-5112 (NX.H6DSV.002),14.990.000đ,acer,15,fhd,intel,5,8,4,16,intel,1tb hdd,win10,2.1


In [58]:
#Bảng trên cho ta thấy được phần Capacity đã lấn hẵn qua cột GPU
df.loc[df['Capacity'] == 16, 'GPU'] = 'uhd 620'
df.loc[df['Capacity'] == 16, 'Storage Type'] = 'hdd'
df.loc[df['Capacity'] == 16, 'Capacity'] = 1000

In [59]:
#Kiểm tra Field Storage Type
df.groupby('Storage Type').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,GPU,OS,Weight
Storage Type,,,,,,,,,,,,,
emmc,2,2,2,2,2,2,2,2,2,2,2,2,2
hdd,295,295,295,295,295,295,295,295,295,295,295,295,295
ssd,237,237,237,237,237,237,237,237,237,237,237,237,237


In [60]:
#Kiểm tra Field GPU
df.groupby('GPU').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,OS,Weight
GPU,,,,,,,,,,,,,
amd radeon vega 3,1,1,1,1,1,1,1,1,1,1,1,1,1
geforce gtx 1650,1,1,1,1,1,1,1,1,1,1,1,1,1
geforce mx130,5,5,5,5,5,5,5,5,5,5,5,5,5
geforce mx250,3,3,3,3,3,3,3,3,3,3,3,3,3
gtx 1050,27,27,27,27,27,27,27,27,27,27,27,27,27
gtx 1050ti,13,13,13,13,13,13,13,13,13,13,13,13,13
gtx 1060,9,9,9,9,9,9,9,9,9,9,9,9,9
gtx 1650,22,22,22,22,22,22,22,22,22,22,22,22,22
gtx 1660ti,12,12,12,12,12,12,12,12,12,12,12,12,12


In [61]:
#Ta thấy có sự không thống nhất trong quá trình nhập liệu
df['GPU'] = df['GPU'].str.replace('amd ', '')
df['GPU'] = df['GPU'].str.replace('geforce ', '')
df['GPU'] = df['GPU'].str.replace('intel ', '')
df['GPU'] = df['GPU'].str.replace(' graphics ', ' ')
df['GPU'] = df['GPU'].str.replace('uhd620', 'uhd 620')
df['GPU'] = df['GPU'].str.replace('1050ti', '1050 ti')
df['GPU'] = df['GPU'].str.replace('1660ti', '1660 ti')

In [63]:
#Kiểm tra Field OS
df.groupby('OS').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,Weight
OS,,,,,,,,,,,,,
endless,1,1,1,1,1,1,1,1,1,1,1,1,1
free dos,66,66,66,66,66,66,66,66,66,66,66,66,66
linux,14,14,14,14,14,14,14,14,14,14,14,14,14
macos,18,18,18,18,18,18,18,18,18,18,18,18,18
ubuntu,9,9,9,9,9,9,9,9,9,9,9,9,9
win 10,1,1,1,1,1,1,1,1,1,1,1,1,1
win10,420,420,420,420,420,420,420,420,420,420,420,420,420
win10 pro,2,2,2,2,2,2,2,2,2,2,2,2,2
windows 10 home sl 64-bit,3,3,3,3,3,3,3,3,3,3,3,3,3


In [64]:
#Có sự không nhất quán 
df.loc[(df['OS'] == 'windows 10 home sl 64-bit'), 'OS'] = 'win10'
df.loc[(df['OS'] == 'win 10'), 'OS'] = 'win10'
df.loc[(df['OS'] == 'linux'), 'OS'] = 'free dos'
df.loc[(df['OS'] == 'ubuntu'), 'OS'] = 'free dos'

In [67]:
#Kiểm tra Field Weight
df.groupby('Weight').count()

,Name,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS
Weight,,,,,,,,,,,,,
0.80,2,2,2,2,2,2,2,2,2,2,2,2,2
1.00,6,6,6,6,6,6,6,6,6,6,6,6,6
1.10,2,2,2,2,2,2,2,2,2,2,2,2,2
1.20,32,32,32,32,32,32,32,32,32,32,32,32,32
1.30,22,22,22,22,22,22,22,22,22,22,22,22,22
1.40,40,40,40,40,40,40,40,40,40,40,40,40,40
1.50,46,46,46,46,46,46,46,46,46,46,46,46,46
1.60,52,52,52,52,52,52,52,52,52,52,52,52,52
1.63,3,3,3,3,3,3,3,3,3,3,3,3,3


In [69]:
#Cuối cùng ta xử lý field Price
df['Price'] = df['Price'].str.replace('.', '')
df['Price'] = df['Price'].str.replace('đ', '')
df['Price'] = df['Price'].astype('float64')
df['Price'] = df['Price'] / 1000

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 14 columns):
Name            534 non-null object
Price           534 non-null float64
Brand           534 non-null object
Screen Size     534 non-null int64
Screen Type     534 non-null object
CPU Brand       534 non-null object
CPU Type        534 non-null int64
CPU Gen         534 non-null int64
Ram             534 non-null int64
Capacity        534 non-null int64
Storage Type    534 non-null object
GPU             534 non-null object
OS              534 non-null object
Weight          534 non-null float64
dtypes: float64(2), int64(5), object(7)
memory usage: 58.5+ KB


In [72]:
#Bỏ đi cột Name của máy vì không cần đến khi đưa vào model
laptop = df.drop('Name', axis=1)

In [73]:
#Lưu lại dữ liệu
df.to_csv("data_laptop.csv", sep='\t', index=False)
laptop.to_csv("data101.csv", sep='\t', index=False)

In [74]:
laptop = pd.read_csv("data101.csv", sep='\t')

In [76]:
#Dữ liệu cuối cùng
laptop.head(20)

,Price,Brand,Screen Size,Screen Type,CPU Brand,CPU Type,CPU Gen,Ram,Capacity,Storage Type,GPU,OS,Weight
0,11990.0,acer,14,fhd,intel,3,10,4,256,ssd,uhd 620,win10,1.6
1,9990.0,acer,15,fhd,intel,3,7,4,256,ssd,hd 620,win10,1.7
2,15999.0,acer,15,fhd,intel,5,8,4,1000,hdd,uhd 620,win10,1.6
3,13499.0,acer,15,fhd,intel,3,8,4,1000,hdd,uhd 620,win10,1.6
4,14499.0,acer,14,fhd,intel,3,8,4,256,ssd,uhd 620,win10,1.5
5,22490.0,acer,15,fhd,intel,5,9,8,512,ssd,gtx 1050,win10,2.3
6,15990.0,acer,15,fhd,intel,5,10,8,512,ssd,uhd 620,win10,1.7
7,17990.0,acer,15,fhd,intel,5,10,8,1000,ssd,uhd 620,win10,1.7
8,21990.0,acer,15,fhd,intel,5,9,8,256,ssd,gtx 1050,win10,2.3
9,18999.0,acer,15,fhd,amd,5,3,8,256,ssd,radeon rx 560x,win10,2.3
